In [21]:
import numpy as np
import skimage
from skimage.io import imread_collection
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
import cv2
from copy import deepcopy
import glob
import cv2 
import os
import pickle
import csv

### Reading trained models

In [24]:
filename = 'fin.sav'
loaded_model = pickle.load(open(filename, 'rb'))
directory = "/Users/krishnabairavi/Downloads/test/"

### Procedure

1. Testing the images on the models. Model takes the amplitude of signal peaks from an image and classifies each amplitude into 0 ( No V's present) and 1 ( V's Present)

2. Once the processed test dataset which contains the amplitude obtained is fed to the model, output label can be predicted and the time at which it occured can be obtained

3. The results are then writen into the csv file " predictions.csv" 

In [25]:
test_set=[]

pred_list = []

for filename in sorted(os.listdir(directory)):
    
    predictions=set()
    
    ## Opening a CSV file to write the outputs in desired format
    
    myFile = open ('predictions.csv','w')
    
    if filename.endswith(".test"): 
        continue
        
    img = mpimg.imread(directory+str (filename))
    
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    total_count = 0
    
    for i in range(10,7500,75):
        
        pic = img[0:75,i:i+75]
        flag = 0
        break_flag = 0
        pic_mod = deepcopy(pic)
        pic_mod = pic_mod/np.linalg.norm(pic_mod)
        sum_pic = np.sum(pic_mod[:,:,2])

    ## Code block to detect blue regions in the annotations ( the one's which are non V's)
    
        if(sum_pic > 43.5 and sum_pic < 45):
            box_number = 0
            
            for m in range(75,750,75):
                pic_m = img[m:m+75,i:i+75]
                pic_mod_b = cv2.cvtColor(pic_m, cv2.COLOR_BGR2GRAY)
                sum_pic_b = np.sum(pic_mod_b)
                box_number += 1
              
                ## To find the black peak in an image using color identification and Harris peak detections
                
                if(sum_pic_b < 5450):
                    
                    operatedImage = np.float32(pic_mod_b)
            
                    dst = cv2.cornerHarris(operatedImage, 2, 5, 0.01)
                    
                    dst_new = cv2.dilate(dst,None)

                    points_unravel=np.unravel_index(dst_new.argmax(),dst.shape)
                    
                    ## Determining the amplitude for every peak
                    
                    value = abs( (((4 - box_number)*75)+points_unravel[0])*0.5 )
        
                    ## Loading the trained model and predicting the class for an amplitude
                    pred= loaded_model.predict (value.reshape(-1,1))
                    
                    ## To determine the time of occurance of V ( which is  predicted class 1 )
                    
                    if pred == [1]:
                        time = (0.2 * i)/75
                        predictions.add(time)
                    
                    break
                    
##Writing the obtained to a csv file 

    for p in predictions:
        pred_list.append([str(filename) + " " + str(p)])

with  myFile:
    writer = csv.writer(myFile)
    writer.writerows(pred_list)
    
    
    
    
    
                    
                    